# **Data Wrangling and Missing Value Handling**

## **Introduction**

### Why is Data Wrangling Important?
- **Real-world data** is often messy, containing missing values, inconsistencies, and irrelevant features.
- **Unprocessed data** can lead to inaccurate insights and unreliable models.
- **Effective data wrangling** transforms raw data into a structured, clean format suitable for analysis and machine learning.

### **Key Steps in Data Wrangling:**
1. **Data Cleaning** – Handling missing values, removing outliers, and standardizing formats.
2. **Data Transformation** – Encoding categorical variables, normalizing numerical data, and feature engineering.
3. **Data Integration** – Merging, reshaping, and aggregating data for better analysis.

## **Dataset Used for Demonstration**
We will use a **synthetic dataset** designed to teach **Data Wrangling** techniques by addressing common data issues. The dataset contains:

### **1. Numerical Features**
- **Age** – Contains missing values and outliers (e.g., unrealistic values).
- **Salary** – Has missing values that require imputation.
- **Work Experience** – Some missing entries, demonstrating handling techniques.
- **Job Satisfaction Score** – Skewed distribution, useful for transformations.
- **Customer Satisfaction Rating** – Ranges from 1 to 10, useful for normalization.

### **2. Categorical Features**
- **Name** – Contains duplicates and inconsistencies (e.g., different cases, extra spaces).
- **Department** – Includes typos and inconsistent categories.
- **Education Level** – Ordinal categorical variable requiring encoding.
- **Remote Work** – Binary categorical feature useful for one-hot encoding.
- **Performance Score** – Imbalanced target variable, demonstrating resampling techniques.

### **3. Date and Currency Fields**
- **Join Date** – Stored in mixed formats, demonstrating date parsing.
- **Bonus** – Contains currency symbols (`$`, `€`), requiring conversion to numeric values.

### **4. Key Data Wrangling Challenges Covered**
- **Handling missing values** in Salary, Work Experience, and Age using imputation techniques.
- **Dealing with outliers** in Age by capping, removing, or transforming values.
- **Fixing inconsistencies** in categorical data through text standardization and deduplication.
- **Encoding categorical variables** for machine learning compatibility.
- **Normalizing and transforming numerical features** to improve data distributions.
- **Addressing imbalanced target variables** through resampling techniques.

This dataset is structured to provide hands-on experience in **Data Wrangling**, helping students learn essential techniques for real-world data preprocessing.


In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import zscore
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from datetime import datetime

# Load the dataset
dataset = pd.read_csv("./data/data_wrangling_dataset.csv")

# Display first few rows
dataset.head()



# **1. Data Cleaning**

## **1.1 Handling Missing Values**

### **Why do Missing Values Occur?**
- **Data collection errors** (e.g., sensor malfunctions, survey non-responses).
- **Human errors** (e.g., incorrect data entry).
- **Different data sources** (some sources may not have certain attributes).

### **Methods to Handle Missing Values:**

1. **Deletion (Dropping Missing Values):**
   - **When to use?** If only a small percentage of data is missing.
   - **Drawback:** Can result in loss of valuable data.

2. **Imputation (Filling Missing Values):**
   - **Mean/Median Imputation** (for numerical data) – works well if data is normally distributed.
   - **Mode Imputation** (for categorical data) – replaces missing values with the most frequent value.
   - **Forward/Backward Fill** (for time-series data) – fills missing values based on previous or next observations.
   - **KNN Imputation** – predicts missing values based on similar data points.


In [ ]:
df = dataset.copy(deep=True)

In [ ]:
# Check missing values
print("Missing values before handling:")
print(df.isnull().sum())

In [ ]:
# Drop rows with missing values (not recommended if data loss is high)
df_dropped = df.dropna()

In [ ]:
# Impute numerical values (Age, Income) using Mean & Median
imputer_mean = SimpleImputer(strategy='mean')
df['Salary'] = imputer_mean.fit_transform(df[['Salary']])

# Predictive Imputation using KNN (for Age & Income)
knn_imputer = KNNImputer(n_neighbors=3)
df[['Salary', 'Work_Experience']] = knn_imputer.fit_transform(df[['Salary', 'Work_Experience']])


# Removing duplicates
df = df.drop_duplicates()

# Check missing values after handling
print("Missing values after handling:")
print(df.isnull().sum())

df.head()


In [ ]:
# Numerical Columns
for col in ['Age', 'Salary', 'Work_Experience']:
    df[col].fillna(df[col].median(), inplace=True)

# Categorical Columns
for col in ['Department', 'Education_Level', 'Remote_Work']:
    df[col].fillna(df[col].mode()[0], inplace=True)

df.head()


## **1.2 Handling Outliers**

### **What are Outliers?**
- **Outliers** are extreme values that differ significantly from other observations.
- They can be caused by **errors** or **natural variations** in the data.

### **Methods to Detect and Remove Outliers:**

1. **Z-Score Method**:
   - Measures how many standard deviations a data point is from the mean.
   - If the absolute Z-score is greater than 3, the value is considered an outlier.

2. **Interquartile Range (IQR) Method**:
   - Detects outliers by identifying values **outside 1.5 times the IQR**.



In [ ]:
# Using Z-Score
z_scores = np.abs(zscore(df['Age']))
df = df[z_scores < 3]  # Removing outliers based on Z-score

# Detecting Outliers using IQR
Q1 = df['Salary'].quantile(0.25)
Q3 = df['Salary'].quantile(0.75)
IQR = Q3 - Q1


df = df[(df['Salary'] >= Q1 - 1.5 * IQR) & (df['Salary'] <= Q3 + 1.5 * IQR)]

df.head()

## **1.4 Standardizing Data Formats**

### **Why Standardization of Data Formats is Important?**

- Ensures Consistency: Data often comes in different formats, making uniform processing essential.

- Facilitates Comparisons: Uniform formats enable accurate analysis and computations.

- Reduces Errors: Inconsistent formats can lead to misinterpretations and processing issues.

- Improves Data Quality: Standardization simplifies data cleaning and validation.

### **Common Data Formats to Standardize**

1. **Date and Time Standardization:** Standardizing date formats ensures accurate sorting, filtering, and time-based analysis. Using a universally accepted format, such as ISO 8601, enhances consistency.

2. **Numeric Data Standardization:** Removing extra characters like currency symbols and ensuring a consistent decimal notation is essential for correct calculations.

3. **Categorical Data Standardization:** Variations in categorical values (e.g., different capitalizations or abbreviations) can cause inconsistencies. Standardizing these values improves reliability.

4. **Text and String Formatting:** Removing unnecessary spaces, special characters, and ensuring uniform capitalization enhances text processing.



In [ ]:
# Standardizing date format
df['Join_Date'] = pd.to_datetime(df['Join_Date'], errors='coerce')

# Converting currency column to numeric using different approaches
df['Bonus'] = df['Bonus'].replace({r'\$': '', r'€': ''}, regex=True).astype(int)

# Standardizing numeric data
df['Salary'] = df['Salary'].astype(int)
df['Job_Satisfaction'] = df['Job_Satisfaction'].astype(int)
df['Work_Experience'] = df['Work_Experience'].astype(int)

df['Name'] = df['Name'].str.strip().str.title()



df.head()


# **2. Data Pre-processing**

## **2.1 Encoding Categorical Variables**

### **Why do we need Encoding?**
- Machine learning models require **numerical data**.
- Encoding converts categorical values into numbers.

### **Types of Encoding:**

1. **Label Encoding**:
   - Assigns **a unique integer** to each category.
   - **Best for** ordinal categorical data (e.g., Small, Medium, Large).

2. **One-Hot Encoding**:
   - Creates separate **binary columns** for each category.
   - **Best for** nominal categorical data (e.g., Cities, Colors).


In [ ]:
# Encoding categorical variables using multiple techniques
label_encoder = LabelEncoder()


# Label Encoding for 'Education_Level' and 'Department'

label_encoder.fit(df["Education_Level"])
education_mapping = dict(zip(df["Education_Level"].unique(), label_encoder.transform(df["Education_Level"].unique())))
df["Education_Level"] = label_encoder.transform(df["Education_Level"])
# df['Education_Level'] = label_encoder.fit_transform(df['Education_Level'])


label_encoder.fit(df["Department"])
department_mapping = dict(zip(df["Department"].unique(), label_encoder.transform(df["Department"].unique())))
df["Department"] = label_encoder.transform(df["Department"])
# df['Department'] = label_encoder.fit_transform(df['Department'])


# One-Hot Encode 'Remote_Work' and drop the original column
df = pd.get_dummies(df, columns=["Remote_Work"], drop_first=True)

print("\nEducation Level Mapping:")
education_df = pd.DataFrame(list(education_mapping.items()), columns=["Original Value", "Encoded Value"])
print(education_df.to_string(index=False))

print("\nDepartment Mapping:")
department_df = pd.DataFrame(list(department_mapping.items()), columns=["Original Value", "Encoded Value"])
print(department_df.to_string(index=False))

df.head()

## **2.1 Handling Imbalanced Datasets**

- **Why It Matters**: In imbalanced datasets, one class significantly outweighs others, leading to biased models that favor the dominant class.
- **Strategies for Handling Imbalance**:
  - **Resampling Methods**: Oversampling the minority class or undersampling the majority class can help balance the dataset.
  - **Synthetic Data Generation**: Techniques like SMOTE (Synthetic Minority Over-sampling Technique) generate synthetic examples to balance class distribution.
  - **Algorithmic Approaches**: Some machine learning models handle imbalance better by adjusting class weights.


In [ ]:
df = df.dropna()

In [ ]:
# Calculate Years at Company
df["Years_at_Company"] = (datetime.now() - df["Join_Date"]).dt.days / 365

# Drop the original Join_Date column
df = df.drop(columns=["Join_Date"])

# Drop irrelevant columns
df = df.drop(columns=["ID", "Name"])  # Keeping only numerical features

# Define features and target
X = df.drop(columns=["Performance_Score"])  # Features
y = df["Performance_Score"]  # Target variable

# Splitting dataset before SMOTE
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Apply SMOTE to balance the classes
smote = SMOTE(sampling_strategy="auto", random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

# Check class distribution after SMOTE
print("Original class distribution:\n", y_train.value_counts())
print("Resampled class distribution:\n", pd.Series(y_resampled).value_counts())

## **2.2 Feature Selection**
- **Why It Matters**: Irrelevant or low-variance features can add noise and reduce model performance.
- **Methods for Feature Selection**:
  - **Variance Thresholding**: Removing features with very low variance helps eliminate those that contribute little to predictive power.
  - **Correlation Analysis**: Highly correlated features can be redundant and may be removed to simplify the model.
  - **Model-Based Selection**: Feature importance

In [ ]:
# Feature selection: Dropping low-variance columns manually

numerical_columns = df.select_dtypes(include=[np.number]).columns


threshold = 0.01 * (1 - 0.01)
low_variance_cols = [col for col in numerical_columns if df[col].var() < threshold]
df.drop(columns=low_variance_cols, inplace=True)

df.head()

## **3. Data Transformation**

## **3.1 Scaling & Normalization**

### **Why Scale Data?**
- Ensures that **all features contribute equally** to the model.
- Improves performance in algorithms like KNN, SVM, and PCA.
- Helps in handling skewed data for better model interpretability.

### **Methods:**
1. **Min-Max Scaling**: Scales data to a range **[0,1]**, preserving relative distances. Useful for models sensitive to feature magnitude.
2. **Standardization (Z-score)**: Centers data around **mean = 0, std = 1**, making it suitable for normally distributed features.
3. **Log Transformation**: Reduces skewness in features with highly skewed distributions, improving model robustness.
4. **Feature Engineering**: Creating new meaningful features, such as ratios, to enhance predictive power.


In [ ]:

# Min-Max Scaling
scaler = MinMaxScaler()
df[['Salary', 'Work_Experience']] = scaler.fit_transform(df[['Salary', 'Work_Experience']])


# Standardization
# df[['Salary', 'Work_Experience']] = StandardScaler().fit_transform(df[['Salary', 'Work_Experience']])

df.head()

In [ ]:
# Log transformation for skewed data using numpy
for col in ['Salary', 'Bonus']:
    df[col] = np.log1p(df[col])

df.head()

In [ ]:
# Feature engineering: Creating new features
df['Experience per Year'] = df['Work_Experience'] / (df['Age'] + 1)

In [ ]:
df.to_csv("./data/cleaned_synthetic_data.csv", index=False)